In [43]:
import math
import torch
def intersection_over_union(boxes_preds, boxes_labels, box_format="midpoint",GIoU=False, UIoU= False, DIoU=False, CIoU=False, AIoU= False, ICIoU=False ,beta =1, eps=1e-7):
    # boxes_preds shape is (N, 4) where N is the number of bboxes
    # bboxes_labels shape is (N, 4)

    if box_format=="midpoint":
        box1_x1 = boxes_preds[..., 0:1] - boxes_preds[..., 2:3] / 2
        box1_y1 = boxes_preds[..., 1:2] - boxes_preds[..., 3:4] / 2
        box1_x2 = boxes_preds[..., 0:1] + boxes_preds[..., 2:3] / 2
        box1_y2 = boxes_preds[..., 1:2] + boxes_preds[..., 3:4] / 2
        box2_x1 = boxes_labels[..., 0:1] - boxes_labels[..., 2:3] / 2
        box2_y1 = boxes_labels[..., 1:2] - boxes_labels[..., 3:4] / 2
        box2_x2 = boxes_labels[..., 0:1] + boxes_labels[..., 2:3] / 2
        box2_y2 = boxes_labels[..., 1:2] + boxes_labels[..., 3:4] / 2

    elif box_format == "corners":
        box1_x1 = boxes_preds[..., 0:1]
        box1_y1 = boxes_preds[..., 1:2]
        box1_x2 = boxes_preds[..., 2:3]
        box1_y2 = boxes_preds[..., 3:4] # (N, 1)
        box2_x1 = boxes_labels[..., 0:1]
        box2_y1 = boxes_labels[..., 1:2]
        box2_x2 = boxes_labels[..., 2:3]
        box2_y2 = boxes_labels[..., 3:4]

    x1 = box1_x1.max(box2_x1)
    y1 = box1_y1.max(box2_y1)
    x2 = box1_x2.min(box2_x2)
    y2 = box1_y2.min(box2_y2)

    w1, h1 = box1_x2 - box1_x1, box1_y2 - box1_y1 + eps
    
    w2, h2 = box2_x2 - box2_x1, box2_y2 - box2_y1 + eps
    
    

    #.clamp(0) is for the case when they donot intersect
    intersection = (x2 - x1).clamp(0) * (y2 - y1).clamp(0)

    box1_area = abs((box1_x2 - box1_x1) * (box1_y1 - box1_y2))
    box2_area = abs((box2_x2 - box2_x1) * (box2_y1 - box2_y2))

    union = box1_area + box2_area - intersection

    iou = intersection/(union + 1e-6)

    if ICIoU or AIoU or CIoU or  DIoU or UIoU or GIoU :
        cw = box1_x2.max(box2_x2) - box1_x1.min(box2_x1)  # convex (smallest enclosing box) width
        ch = box1_y2.max(box2_y2) - box1_y1.min(box2_y1)  # convex height

        c2 = cw.pow(2) + ch.pow(2) + eps  # convex diagonal squared
            
        rho2 = (
                (box2_x1 + box2_x2 - box1_x1 - box1_x2).pow(2) + (box2_y1 + box2_y2 - box1_y1 - box1_y2).pow(2))/ 4  # center dist**2
        
        if CIoU or DIoU or ICIoU:  # Distance or Complete IoU https://arxiv.org/abs/1911.08287v1
            
            if CIoU:  # https://github.com/Zzh-tju/DIoU-SSD-pytorch/blob/master/utils/box/box_utils.py#L47
                v = (4 / math.pi**2) * ((w2 / h2).atan() - (w1 / h1).atan()).pow(2)
                
                with torch.no_grad():
                    alpha = v / (v - iou + (1 + eps))
                if AIoU:
                    A_Agt2 = (box1_x1 - box2_x1).pow(2) + (box1_y1 - box2_y1).pow(2)
                    B_Bgt2 = (box1_x2 - box2_x2).pow(2) + (box1_y2 - box2_y2).pow(2)
                    return iou - (rho2 / c2) + (v * alpha) + beta * ((A_Agt2 + B_Bgt2)/c2)   # AIoU
                
                return iou - (rho2 / c2) + (v * alpha)  # CIoU
            elif ICIoU:
                vv = (8 / math.pi**2) * (((w2 / w1).atan() - (math.pi/4)).pow(2)  - ((h2 / h1).atan() - (math.pi/4)).pow(2))
                
                with torch.no_grad():
                    alpha = vv / (vv - iou + (1 + eps))
                return iou - (rho2 / c2 + vv * alpha)  # ICIoU
        
                
            return iou - (rho2 / c2)  # DIoU
        c_area = cw * ch + eps  # convex area
        giou = iou - (c_area - union) / c_area  # GIoU https://arxiv.org/pdf/1902.09630.pdf
        if UIoU:
            normalized_distance = c2/rho2

            similarity = math.sqrt(min(box1_area, box1_area) / max(box1_area, box1_area)) if max(box1_area, box1_area) > 0 else 0
            if iou == 0:  # Non-overlapping case
                uiou = 0 + 0.5 * (1 - normalized_distance)
            elif iou > 0 and giou < 0.98:  # Partial overlap
                uiou = 0.5 + 0.48 * (1 + giou) / 2
            else:  # One box inside another
                uiou = 0.98 + 0.02 * ((1 / similarity ** 2) + (1 - normalized_distance)) / 2
        
            return uiou 
            
        return giou
    return iou  # IoU

In [44]:
box1 = torch.tensor([1,1,3,3])
box2 = torch.tensor([2,2,4,4])

In [45]:
iou = intersection_over_union(box1, box2, box_format="corners",)
iou

tensor([0.1429])

In [46]:
diou = intersection_over_union(box1, box2, box_format="corners",DIoU=True)
diou

tensor([0.0317])

In [47]:
giou = intersection_over_union(box1, box2, box_format="corners",GIoU=True)
giou

tensor([-0.0794])

In [48]:
ciou = intersection_over_union(box1, box2, box_format="corners",CIoU=True)
ciou

tensor([0.0317])

In [49]:
iciou = intersection_over_union(box1, box2, box_format="corners",ICIoU=True)
iciou

tensor([0.0317])

In [50]:
aiou = intersection_over_union(box1, box2, box_format="corners",AIoU=True)
aiou

tensor([-0.0794])

In [53]:
uiou = intersection_over_union(box1, box2, box_format="corners",UIoU=True)
uiou

tensor([0.7210])